<a href="https://colab.research.google.com/github/IMIYOU/gardendream/blob/master/DFL2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 欢迎使用DFL2.0英文版!

这是针对Google Colab的DFL的改编版本,如使用中出现问题可联系QQ：269777955！


# 概述
*   提取器功能齐全！
*   训练不可以使用预览！
*   合并功能完整！
*   您可以使用Google驱动器导入/导出工作区！
*   使用“管理工作区”块中可以执行的工作区导入/导出和其他操作！
*   Google Colab机器激活12小时，DFL Colab在训练模式下备份您的工作区！
*   谷歌不喜欢长期的繁重计算！因此，对于连续两次以上的训练，使用两个Google帐户！建议将您的训练分成两个帐户，但您可以使用一个谷歌驱动器帐户来存储您的工作区！


# 查看分配到的GPU

*   Google Colab可以为您提供一个特斯拉图形卡: K80, T4, P4 or P100
*   在使用DeepFaceLab之前，您可以在这里检查GPU的型号



In [0]:
!nvidia-smi

#挂载Google云盘

In [0]:
#@title 开始挂载 
#挂载谷歌云盘
#点击链接授权，复制授权码，填入框框，然后回车。

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

#创建DeepFaceLab目录，并且进入目录
%cd /content/drive/My Drive/
!mkdir DeepFaceLab
%cd /content/drive/My Drive/DeepFaceLab/


# 安装DFL程序以及依赖环境

* 全新安装或者升级现有程序
* 所有安装都是自动的

In [0]:
#@title 安装DeepFaceLab程序

Mode = "全新安装" #@param ["全新安装", "更新程序"]

from pathlib import Path
if (Mode == "全新安装"):
  !git clone 

  # fix linux warning
  # /usr/lib/python3.6/multiprocessing/semaphore_tracker.py:143: UserWarning: semaphore_tracker: There appear to be 1 leaked semaphores to clean up at shutdown
  fin = open("/usr/lib/python3.6/multiprocessing/semaphore_tracker.py", "rt")
  data = fin.read()
  data = data.replace('if cache:', 'if False:')
  fin.close()

  fin = open("/usr/lib/python3.6/multiprocessing/semaphore_tracker.py", "wt")
  fin.write(data)
  fin.close()
else:
  %cd /content/drive/My Drive/DeepFaceLab/DeepFaceLab/
  !git pull

if not Path("/content/drive/My\ Drive/DeepFaceLab/pretrain").exists():
  print("Downloading CelebA faceset ... ")
  !wget -q --no-check-certificate -r 'https://github.com/chervonij/DFL-Colab/releases/download/pretrain-CelebA/pretrain_CelebA.zip' -O /content/drive/My\ Drive/DeepFaceLab/pretrain_CelebA.zip
  !mkdir /content/drive/My\ Drive/DeepFaceLab/pretrain
  !unzip -q /content/drive/My\ Drive/DeepFaceLab/pretrain_CelebA.zip -d /content/drive/My\ Drive/DeepFaceLab/pretrain/
  !rm /content/drive/My\ Drive/DeepFaceLab/pretrain_CelebA.zip

if not Path("/content/drive/My\ Drive/DeepFaceLab/pretrain_Q96").exists():
  print("Downloading Q96 pretrained model ...")
  !wget -q --no-check-certificate -r 'https://github.com/chervonij/DFL-Colab/releases/download/Q96_model_pretrained/Q96_model_pretrained.zip' -O /content/drive/My\ Drive/DeepFaceLab/pretrain_Q96.zip
  !mkdir /content/drive/My\ Drive/DeepFaceLab/pretrain_Q96
  !unzip -q /content/drive/My\ Drive/DeepFaceLab/pretrain_Q96.zip -d /content/drive/My\ Drive/DeepFaceLab/pretrain_Q96/
  !rm /content/drive/My\ Drive/DeepFaceLab/pretrain_Q96.zip

if not Path("/content/drive/My\ Drive/DeepFaceLab/workspace").exists():
  !mkdir /content/drive/My\ Drive/DeepFaceLab/workspace; mkdir /content/drive/My\ Drive/DeepFaceLab/workspace/data_src; mkdir /content/drive/My\ Drive/DeepFaceLab/workspace/data_src/aligned; mkdir /content/drive/My\ Drive/DeepFaceLab/workspace/data_dst; mkdir /content/drive/My\ Drive/DeepFaceLab/workspace/data_dst/aligned; mkdir /content/drive/My\ Drive/DeepFaceLab/workspace/model  

print("Done!")

In [0]:
#@title 安装Python依赖
!pip uninstall -y tensorflow
!pip install tqdm
!pip install numpy==1.17.0
!pip install h5py==2.9.0
!pip install opencv-python==4.1.0.25
!pip install ffmpeg-python==0.1.17
!pip install scikit-image==0.14.2
!pip install scipy==1.4.1
!pip install colorama
!pip install tensorflow-gpu==1.14.0
!pip install --upgrade scikit-image
!apt-get install cuda-10-0

#管理工作区



*   您可以导入/导出工作区或单个数据，如使用Google Drive的模型文件
*   此外，还可以使用HFS（HTTP文件服务器）从计算机直接导入/导出工作区
*   您可以清除所有工作区或删除其中的一部分



In [0]:
#@title 从云盘根目录解压文件到工作区

Mode = "models" #@param ["workspace", "data_src", "data_dst", "data_src aligned", "data_dst aligned", "models"]
Archive_name = "model.zip" #@param {type:"string"}

#Mount Google Drive as folder
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

def zip_and_copy(path, mode):
  unzip_cmd=" -q "+Archive_name
  
  %cd $path
  copy_cmd = "/content/drive/My\ Drive/"+Archive_name+" "+path
  !cp $copy_cmd
  !unzip $unzip_cmd    
  !rm $Archive_name

if Mode == "workspace":
  zip_and_copy("/content/drive/My\ Drive/DeepFaceLab", "workspace")
elif Mode == "data_src":
  zip_and_copy("/content/drive/My\ Drive/DeepFaceLab/workspace", "data_src")
elif Mode == "data_dst":
  zip_and_copy("/content/drive/My\ Drive/DeepFaceLab/workspace", "data_dst")
elif Mode == "data_src aligned":
  zip_and_copy("/content/drive/My\ Drive/DeepFaceLab/workspace/data_src", "aligned")
elif Mode == "data_dst aligned":
  zip_and_copy("/content/drive/My\ Drive/DeepFaceLab/workspace/data_dst", "aligned")
elif Mode == "models":
  zip_and_copy("/content/drive/My\ Drive/DeepFaceLab/workspace", "model")
  
print("Done!")



In [0]:
#@title 压缩工作区文件到云盘根目录 { form-width: "30%" }
Mode = "data_src" #@param ["workspace", "data_src", "data_dst", "data_src aligned", "data_dst aligned", "merged", "models", "result video"]
Archive_name = "data_src.zip" #@param {type:"string"}

#Mount Google Drive as folder
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

def zip_and_copy(path, mode):
  zip_cmd="-r -q "+Archive_name+" "
  
  %cd $path
  zip_cmd+=mode
  !zip $zip_cmd
  copy_cmd = " "+Archive_name+"  /content/drive/My\ Drive/"
  !cp $copy_cmd
  !rm $Archive_name

if Mode == "workspace":
  zip_and_copy("/content/drive/My\ Drive/DeepFaceLab", "workspace")
elif Mode == "data_src":
  zip_and_copy("/content/drive/My\ Drive/DeepFaceLab/workspace", "data_src")
elif Mode == "data_dst":
  zip_and_copy("/content/drive/My\ Drive/DeepFaceLab/workspace", "data_dst")
elif Mode == "data_src aligned":
  zip_and_copy("/content/drive/My\ Drive/DeepFaceLab/workspace/data_src", "aligned")
elif Mode == "data_dst aligned":
  zip_and_copy("/content/drive/My\ Drive/DeepFaceLab/workspace/data_dst", "aligned")
elif Mode == "merged":
  zip_and_copy("/content/drive/My\ Drive/DeepFaceLab/workspace/data_dst", "merged")
elif Mode == "models":
  zip_and_copy("/content/drive/My\ Drive/DeepFaceLab/workspace", "model")
elif Mode == "result video":
  !cp /content/drive/My\ Drive/DeepFaceLab/workspace/result.mp4 /content/drive/My\ Drive/
  
print("Done!")


In [0]:
#@title 从HTTP文件服务器解压文件到工作区{ form-width: "30%", display-mode: "form" }
URL = "http://" #@param {type:"string"}
Mode = "download to workspace" #@param ["unzip to DeepFaceLab", "unzip to workspace", "unzip to workspace/data_src", "unzip to workspace/data_src/aligned", "unzip to workspace/data_dst", "unzip to workspace/data_dst/aligned", "unzip to workspace/model", "download to workspace"]

import urllib
from pathlib import Path

def unzip(zip_path, dest_path):

    
  unzip_cmd = " unzip -q " + zip_path + " -d "+dest_path
  !$unzip_cmd  
  rm_cmd = "rm "+dest_path + url_path.name
  !$rm_cmd
  print("Unziped!")
  

if Mode == "unzip to DeepFaceLab":
  dest_path = "/content/drive/My\ Drive/DeepFaceLab/"
elif Mode == "unzip to workspace":
  dest_path = "/content/drive/My\ Drive/DeepFaceLab/workspace/"
elif Mode == "unzip to workspace/data_src":
  dest_path = "/content/drive/My\ Drive/DeepFaceLab/workspace/data_src/"
elif Mode == "unzip to workspace/data_src/aligned":
  dest_path = "/content/drive/My\ Drive/DeepFaceLab/workspace/data_src/aligned/"
elif Mode == "unzip to workspace/data_dst":
  dest_path = "/content/drive/My\ Drive/DeepFaceLab/workspace/data_dst/"
elif Mode == "unzip to workspace/data_dst/aligned":
  dest_path = "/content/drive/My\ Drive/DeepFaceLab/workspace/data_dst/aligned/"
elif Mode == "unzip to workspace/model":
  dest_path = "/content/drive/My\ Drive/DeepFaceLab/workspace/model/"
elif Mode == "download to workspace":
  dest_path = "/content/drive/My\ Drive/DeepFaceLab/workspace/"

if not Path("/content/drive/My\ Drive/DeepFaceLab/workspace").exists():
  cmd = "mkdir /content/drive/My\ Drive/DeepFaceLab/workspace; mkdir /content/drive/My\ Drive/DeepFaceLab/workspace/data_src; mkdir /content/drive/My\ Drive/DeepFaceLab/workspace/data_src/aligned; mkdir /content/drive/My\ Drive/DeepFaceLab/workspace/data_dst; mkdir /content/drive/My\ Drive/DeepFaceLab/workspace/data_dst/aligned; mkdir /content/drive/My\ Drive/DeepFaceLab/workspace/model"
  !$cmd

url_path = Path(URL)
urllib.request.urlretrieve ( URL, dest_path + url_path.name )

if (url_path.suffix == ".zip") and (Mode!="download to content/drive/My\ Drive/DeepFaceLab/workspace"):
  unzip(dest_path + url_path.name, dest_path)

  
print("Done!")

In [0]:
#@title 从工作区压缩文件到HTTP文件服务器
URL = "http://" #@param {type:"string"}
Mode = "upload workspace" #@param ["upload workspace", "upload data_src", "upload data_dst", "upload data_src aligned", "upload data_dst aligned", "upload merged", "upload model", "upload result video"]

cmd_zip = "zip -r -q "

def run_cmd(zip_path, curl_url):
  cmd_zip = "zip -r -q "+zip_path
  cmd_curl = "curl --silent -F "+curl_url+" -D out.txt > /dev/null"
  !$cmd_zip
  !$cmd_curl


if Mode == "upload workspace":
  %cd "/content"
  run_cmd("workspace.zip workspace/","'data=@/content/drive/My\ Drive/workspace.zip' "+URL)
elif Mode == "upload data_src":
  %cd "/content/workspace"
  run_cmd("data_src.zip data_src/", "'data=@/content/drive/My\ Drive/workspace/data_src.zip' "+URL)
elif Mode == "upload data_dst":
  %cd "/content/workspace"
  run_cmd("data_dst.zip data_dst/", "'data=@/content/drive/My\ Drive/workspace/data_dst.zip' "+URL)
elif Mode == "upload data_src aligned":
  %cd "/content/workspace"
  run_cmd("data_src_aligned.zip data_src/aligned", "'data=@/content/drive/My\ Drive/workspace/data_src_aligned.zip' "+URL )
elif Mode == "upload data_dst aligned":
  %cd "/content/workspace"
  run_cmd("data_dst_aligned.zip data_dst/aligned/", "'data=@/content/drive/My\ Drive/workspace/data_dst_aligned.zip' "+URL)
elif Mode == "upload merged":
  %cd "/content/workspace/data_dst"
  run_cmd("merged.zip merged/","'data=@/content/drive/My\ Drive/workspace/data_dst/merged.zip' "+URL )
elif Mode == "upload model":
  %cd "/content/workspace"
  run_cmd("model.zip model/", "'data=@/content/drive/My\ Drive/workspace/model.zip' "+URL)
elif Mode == "upload result video":
  %cd "/content/workspace"
  run_cmd("result.zip result.mp4", "'data=@/content/drive/My\ Drive/workspace/result.zip' "+URL)
  
  
!rm *.zip

%cd "/content/drive/My\ Drive"
print("Done!")

In [0]:
#@title 删除并重建工作区目录
Mode = "Delete and recreate workspace" #@param ["Delete and recreate workspace", "Delete models", "Delete data_src", "Delete data_src aligned", "Delete data_src video", "Delete data_dst", "Delete data_dst aligned", "Delete merged frames"]

%cd "/content/drive/My Drive/DeepFaceLab" 

if Mode == "Delete and recreate workspace":
  cmd = "rm -r /content/drive/My\ Drive/DeepFaceLab/workspace ; mkdir /content/drive/My\ Drive/DeepFaceLab/workspace; mkdir /content/drive/My\ Drive/DeepFaceLab/workspace/data_src; mkdir /content/drive/My\ Drive/DeepFaceLab/workspace/data_src/aligned; mkdir /content/drive/My\ Drive/DeepFaceLab/workspace/data_dst; mkdir /content/drive/My\ Drive/DeepFaceLab/workspace/data_dst/aligned; mkdir /content/drive/My\ Drive/DeepFaceLab/workspace/model"  
elif Mode == "Delete models":
  cmd = "rm -r /content/drive/My\ Drive/DeepFaceLab/workspace/model/*"
elif Mode == "Delete data_src":
  cmd = "rm /content/drive/My\ Drive/DeepFaceLab/workspace/data_src/*.png || rm -r /content/drive/My\ Drive/DeepFaceLab/workspace/data_src/*.jpg"
elif Mode == "Delete data_src aligned":
  cmd = "rm -r /content/drive/My\ Drive/DeepFaceLab/workspace/data_src/aligned/*"
elif Mode == "Delete data_src video":
  cmd = "rm -r /content/drive/My\ Drive/DeepFaceLab/workspace/data_src.*"
elif Mode == "Delete data_dst":
  cmd = "rm /content/drive/My\ Drive/DeepFaceLab/workspace/data_dst/*.png || rm /content/drive/My\ Drive/DeepFaceLab/workspace/data_dst/*.jpg"
elif Mode == "Delete data_dst aligned":
  cmd = "rm -r /content/drive/My\ Drive/DeepFaceLab/workspace/data_dst/aligned/*"
elif Mode == "Delete merged frames":
  cmd = "rm -r /content/drive/My\ Drive/DeepFaceLab/workspace/data_dst/merged"
  
!$cmd
print("Done!")

# 提取、排序和人脸素材工具
* 为SRC或DST视频提取帧
* 对SRC或DST视频进行去噪
* 使用其中一个检测器提取人脸，（S3FD-推荐），如果需要，可以使用修复提取帧
* 提取并手动排序后将工作区导出到Google Drive（在“管理工作区”模块中）
* 你可以使用DFL FacesetEnhancer增强你的faceset增强图像将取代您的“aligned”目录


In [0]:
#@title 视频转图片
Video = "data_src" #@param ["data_src", "data_dst"]

%cd "/content/drive/My Drive/DeepFaceLab"

cmd = "DeepFaceLab/main.py videoed extract-video"

if Video == "data_dst":
  cmd+= " --input-file workspace/data_dst.* --output-dir workspace/data_dst/"
else:
  cmd+= " --input-file workspace/data_src.* --output-dir workspace/data_src/"
  
!python $cmd

In [0]:
#@title 图片去噪
Data = "data_src" #@param ["data_src", "data_dst"]
Factor = 1 #@param {type:"slider", min:1, max:20, step:1}

cmd = "DeepFaceLab/main.py videoed denoise-image-sequence --input-dir workspace/"+Data+" --factor "+str(Factor)

%cd "/content/drive/My Drive/DeepFaceLab"
!python $cmd

In [0]:
#@title 人脸素材提取
Data = "data_src" #@param ["data_src", "data_dst"]
Detector = "S3FD" #@param ["S3FD", "S3FD (整张脸)"]
Debug = False #@param {type:"boolean"}

detect_type = "s3fd"
dbg = " --output-debug" if Debug else " --no-output-debug"

folder = "workspace/"+Data
folder_aligned = folder+"/aligned"

cmd = "DeepFaceLab/main.py extract --input-dir "+folder+" --output-dir "+folder_aligned
cmd+=" --detector "+detect_type+" --force-gpu-idxs 0"+dbg

if "整张脸" in Detector:
  cmd+=" --face-type whole_face" 
%cd "/content/drive/My Drive/DeepFaceLab"
!python $cmd

In [0]:
#@title 人脸排序检查
Data = "data_src" #@param ["data_src", "data_dst"]
sort_type = "hist" #@param ["blur", "face-yaw", "face-pitch", "face-source-rect-size", "hist", "hist-dissim", "brightness", "hue", "black", "origname", "oneface", "final", "final-faster", "absdiff"]

cmd = "DeepFaceLab/main.py sort --input-dir workspace/"+Data+"/aligned --by "+sort_type

%cd "/content/drive/My Drive/DeepFaceLab"
!python $cmd

In [0]:
#@title 人脸素材细节提升
Data = "data_src" #@param ["data_src", "data_dst"]

data_path = "/content/drive/My\ Drive/DeepFaceLab/workspace/"+Data+"/aligned"
cmd = "/content/drive/My\ Drive/DeepFaceLab/DeepFaceLab/main.py facesettool enhance --input-dir "+data_path
!python $cmd

In [0]:
#@title 人脸素材打包
Data = "data_src" #@param ["data_src", "data_dst"]

data_path = "/content/drive/My\ Drive/DeepFaceLab/workspace/"+Data+"/aligned"
cmd = "/content/drive/My\ Drive/DeepFaceLab/DeepFaceLab/main.py util --pack-faceset --input-dir "+data_path
!python $cmd

In [0]:
#@title 人脸素材解包
Data = "data_src" #@param ["data_src", "data_dst"]

data_path = "/content/drive/My\ Drive/DeepFaceLab/workspace/"+Data+"/aligned"
cmd = "/content/drive/My\ Drive/DeepFaceLab/DeepFaceLab/main.py util --unpack-faceset --input-dir "+data_path
!python $cmd

In [0]:
#@title 为素材应用或者移除已经训练好的XSeg遮罩（记得解包人脸素材后操作）
Mode = "Remove mask" #@param ["Apply mask", "Remove mask"]
Data = "data_src" #@param ["data_src", "data_dst"]

main_path = "/content/drive/My\ Drive/DeepFaceLab/DeepFaceLab/main.py "
data_path = "/content/drive/My\ Drive/DeepFaceLab/workspace/"+Data+"/aligned "
mode_arg = "apply " if Mode == "Apply mask" else "remove "
cmd = main_path+"xseg "+mode_arg+"--input-dir "+data_path
cmd += "--model-dir /content/drive/My\ Drive/DeepFaceLab/workspace/model" if mode_arg == "apply " else ""

!python $cmd

# 训练模式

* 训练模式分为Quick96和SAEHD两个模型以及XSeg模型遮罩方案，根据自己需求选择！
* 在输出字段上设置模型选项
* 如果转到文件管理器中的“模型”文件夹并双击preview.jpg文件，则可以手动查看预览
* 此外，还可以在“管理工作区”块中手动导出工作区，训练XSeg模型时要与你的SAEHD模型训练范围相同！

In [0]:
#@title 模型训练
#挂载谷歌云盘
#点击链接授权，复制授权码，填入框框，然后回车。

Model = "SAEHD" #@param ["SAEHD", "Quick96", "XSeg"]
Silent_Start = True #@param {type:"boolean"}

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# 进入DeepFaceLab_Colab目录
%cd /content/drive/My Drive/DeepFaceLab/DeepFaceLab

# 安装Python依赖
!pip uninstall -y tensorflow
!pip install tqdm
!pip install numpy==1.17.0
!pip install h5py==2.9.0
!pip install opencv-python==4.1.0.25
!pip install ffmpeg-python==0.1.17
!pip install scikit-image==0.14.2
!pip install scipy==1.4.1
!pip install colorama
!pip install tensorflow-gpu==1.14.0
!pip install --upgrade scikit-image
!apt-get install cuda-10-0

# 开始训练SAEHD
cmd ="main.py train --training-data-src-dir ../workspace/data_src/aligned --training-data-dst-dir ../workspace/data_dst/aligned --model-dir ../workspace/model --model " + Model +" --no-preview"

if Model == "Quick96":
  cmd+= " --pretrained-model-dir pretrain_Q96"

if Silent_Start:
  cmd+= " --silent-start"

else:
  train_cmd = (cmd)

!python $cmd

# 合成处理

In [0]:
#@title 合成图片
Model = "SAEHD" #@param ["SAEHD", "Quick96" ]

cmd = "DeepFaceLab/main.py merge --input-dir workspace/data_dst --output-dir workspace/data_dst/merged --output-mask-dir workspace/data_dst/merged_mask --aligned-dir workspace/data_dst/aligned --model-dir workspace/model --model "+Model

%cd "/content/drive/My Drive/DeepFaceLab"
!python $cmd

In [0]:
#@title 合成视频并复制到Google 云盘
Mode = "result video" #@param ["result video", "result_mask video"]
Copy_to_Drive = True #@param {type:"boolean"}


if Mode == "result video":
  !python DeepFaceLab/main.py videoed video-from-sequence --input-dir workspace/data_dst/merged --output-file workspace/result.mp4 --reference-file workspace/data_dst.mp4 --include-audio
  if Copy_to_Drive:
    !cp /content/drive/My\ Drive/DeepFaceLab/workspace/result.mp4 /content/drive/My\ Drive/
elif Mode == "result_mask video":
  !python DeepFaceLab/main.py videoed video-from-sequence --input-dir workspace/data_dst/merged_mask --output-file workspace/result_mask.mp4 --reference-file workspace/data_dst.mp4
  if Copy_to_Drive:
    !cp /content/drive/My\ Drive/DeepFaceLab/workspace/result_mask.mp4 /content/drive/My\ Drive/


#其它工具

In [0]:
#@title 创建驱动器工作区模型目录并移动模型文件到驱动器
%cd /content/workspace/
!mkdir model
!cp /content/drive/My Drive/DeepFaceLab/workspace/model/*.* /content/workspace/model/